# Quantum Noise & Transpilation — Detailed Notes (Session 6)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  

> **Purpose.**  These notes expand the Session-6 slides into a stand-alone reference. You will learn how to model the dominant noise channels on NISQ hardware, apply error-mitigation tactics, and use Qiskit’s transpiler to shorten and noise-adapt circuits.  Mini-exercises with concise solutions appear at the end.

---

## Session Road-map  

1. Quick recap: Grover + ideal assumptions  
2. Nature & taxonomy of quantum noise  
3. Noise models (Pauli, depolarising, thermal, coherent)  
4. Error-mitigation techniques (ZNE, read-out, PEC, DD)  
5. Transpilation: goals, passes, optimisation levels  
6. Qiskit demos: importing a real device noise model, simulating noisy Grover  
7. Impact on AI / DS workloads  
8. Q&A  

---

## 0) Recap — where we stand  

| Algorithm | Ideal query complexity | Real-device caveat |
|-----------|-----------------------|--------------------|
| Grover | $O(\sqrt N)$ oracle calls | Requires ~$\sqrt N$ *iterations* → depth grows; noise may erase advantage. |
| Key vulnerability | Depth & multi-qubit gates accumulate errors; diffusion uses many CNOTs. |

**Today’s goal**: quantify that vulnerability and learn mitigation / transpilation tactics.

---

## 1) What is quantum noise?  

**Definition.** Any uncontrolled interaction or imperfect control operation that drives the system away from its intended unitary evolution.

| Source | Phenomenon | Symbol |
|--------|------------|--------|
| Environment coupling | **Decoherence** (T$_1$ relaxation, T$_2$ dephasing) | $T_1, T_2$ |
| Gate control error | **Coherent** over/under-rotation, crosstalk | $\delta\theta$ |
| Stochastic events | **Pauli flips** $X,Y,Z$ | $p_X,p_Y,p_Z$ |
| Read-out | Bit-flip at measurement | confusion matrix |

Noise turns pure states into *mixed* states; mathematically described by *quantum channels* (CPTP maps) with Kraus operators $\{K_i\}$.

---

## 2) Canonical noise models  

### 2.1 Pauli channel  
$$
\mathcal E(\rho) = (1-p)\rho + p_X X\rho X + p_Y Y\rho Y + p_Z Z\rho Z.
$$

### 2.2 Depolarising channel  
$$
\mathcal D_p(\rho) = (1-p)\rho + p \frac{I}{2^n}.
$$
Uniformly randomises state with prob $p$.

### 2.3 Thermal relaxation (T$_1$/T$_2$)  
Time-dependent Kraus operators derived from Lindblad master equation; Qiskit provides `thermal_relaxation_error(T1, T2, gate_time)`.

### 2.4 Coherent over-rotation  
Ideal rotation $e^{-i\theta X/2}$ executed as $e^{-i(\theta+\delta) X/2}$ — systematic, **not** stochastic.

---

## 3) Impact on algorithm fidelity  

- Error probability per two-qubit gate (CNOT) on today’s IBM devices: $\epsilon_{\text{CNOT}}\sim 1\%-2\%$.  
- Grover with $k$ iterations uses $O(k n)$ CNOTs; fidelity roughly $(1-\epsilon_{\text{CNOT}})^{k n}$.  
  For $n=6, k=8$ and $\epsilon=0.02$, global success ≤ $0.02$ — advantage gone.

*Observation*: **depth minimisation** is crucial; hence good transpilation and mitigation.

---

## 4) Error-mitigation toolbox (no code-distance overhead)

| Technique | Idea | Qiskit tool |
|-----------|------|-------------|
| Zero-Noise Extrapolation (ZNE) | Stretch/deepen circuit (e.g., pulse stretching, gate folding) → fit curve → extrapolate to zero noise. | `qiskit.algorithms.ZNE` (Mitiq) |
| Read-out mitigation | Calibrate $2\times2$ confusion matrix per qubit; apply inverse. | `qiskit.ignis.mitigation` |
| Probabilistic error cancellation (PEC) | Sample inverse-noise operations weighted by Quasi-probabilities. | `qiskit.algorithms.PEC` |
| Dynamical decoupling (DD) | Insert idle-time $\pi$ pulses to average out low-freq noise. | `transpile(..., scheduling_method='dynamic', dd_sequence=...)` |

> **Limitation**: mitigation boosts *effective* shots, runtime, or variance; not a replacement for fault-tolerance.

---

## 5) Transpilation fundamentals  

**Goal**: Adjust circuit to hardware constraints while **reducing** error accumulation.

### Steps  
1. **Layout**: map logical qubits to physical qubits with highest coherence & connectivity (noise-adaptive).  
2. **Routing**: insert SWAPs to satisfy coupling graph.  
3. **Decomposition**: break gates into native set (e.g., U3 + CX).  
4. **Optimisation**: commute, cancel, merge single-qubit rotations, pull gates across SWAPs.

### Optimisation levels (Qiskit)  

| Level | What happens | When to use |
|-------|--------------|-------------|
| 0 | Minimal: map + basis translation | Quick sanity checks |
| 1 | Basic gate-merge & CX cancellation | Low compile time |
| 2 | Heavier commutation + noise-aware layout | Good default |
| 3 | Exhaustive peephole; may alter structure | When every fraction of fidelity counts |

---

### Example: transpiling Grover for `ibm_cairo` (connectivity heavy hex)

```python
from qiskit import QuantumCircuit, transpile, IBMQ
from qiskit.providers.fake_provider import FakeCairoV2

backend = FakeCairoV2()      # offline mock backend incl. noise data
grover_ideal = ...           # build Grover circuit as in Session 5
grover_opt = transpile(grover_ideal,
                       backend=backend,
                       optimization_level=2)
print("Orig depth:", grover_ideal.depth(),
      "→ transpiled depth:", grover_opt.depth(),
      "CX count:", grover_opt.count_ops().get('cx',0))
```

---

## 6) Simulating noisy back-ends  

```python
from qiskit_aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeCairoV2
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

device = FakeCairoV2()
noise_model = NoiseModel.from_backend(device)
sim = AerSimulator(noise_model=noise_model,
                   basis_gates=noise_model.basis_gates,
                   coupling_map=device.configuration().coupling_map)

result_ideal = AerSimulator().run(grover_opt, shots=2000).result()
result_noisy = sim.run(grover_opt, shots=2000).result()

plot_histogram([result_ideal.get_counts(),
                result_noisy.get_counts()],
               legend=['ideal','noisy'])
plt.show()
```

Compute *fidelity* (overlap) by `qiskit.quantum_info.Statevector(grover_opt).evolve(...)` versus noisy density matrix.

---

## 7) AI / DS implications  

| Task | Noise challenge | Mitigation / transpilation benefit |
|------|-----------------|------------------------------------|
| QSVM kernels | Need accurate overlap estimates; depolarising noise biases similarity. | Read-out mitigation & DD improve kernel matrix condition. |
| Variational QNNs | Parameter updates may chase noise rather than landscape (barren plateau). | Short-depth ansätze & level-3 transpilation reduce gradient dilution. |
| Grover-style feature search | Amplitude amplification degraded by phase errors. | ZNE + repeated short-depth Grover with majority vote. |

> **Research direction**: noise-aware training (optimise parameters on noisy model directly).

---

## 8) Worked example — Zero-Noise Extrapolation on 3-qubit Grover  

```python
from mitiq.zne import execute_with_zne, folding

fold_func = folding.fold_gates_at_random
def execute(circuit, shots=2048):
    return sim.run(transpile(circuit, backend=sim,
                             optimization_level=2),
                   shots=shots).result().data()['counts']['101'] / shots

estimated_prob = execute_with_zne(grover_opt, execute,
                                  scale_noise=fold_func, num_to_average=3)
print("ZNE-extrapolated success:", round(estimated_prob,3))
```

---

## 9) Mini-exercises (answers in Appendix)

1. **T$_1$/T$_2$**: Derive Kraus set for pure dephasing ($T_2$).  
2. Build a 2-qubit circuit $H\otimes I → CX → H\otimes I$. Transpile for `FakeJakarta` at optimisation levels 0 and 3; compare depth & CX count.  
3. Apply read-out mitigation to a 4-qubit parity circuit and measure improvement in accuracy.  
4. For Grover with $n=4$ (depth ~80), estimate success probability given per-gate depolarising error $p=0.005$.  
5. Show that coherent over-rotation error of angle $\delta$ each Grover iteration shifts optimal $k$. Compute first-order correction.

---

## 10) FAQ / common pitfalls  

- **“Optimisation level 3 made my circuit *longer*?”** Heavier passes sometimes trade CX count vs. 1-qubit rotations; cost function dependent.  
- **“Noise model import fails.”** Ensure backend supports `backend.properties()`. Use `FakeProvider` if no online access.  
- **“Error-mitigation code explodes shots.”** PEC increases variance → need more shots; factor that into runtime budgeting.  
- **“Transpiler swaps my qubits.”** Use `initial_layout` or logical names to lock mapping if necessary.

---

## 11) Summary (Session 6)

- **Noise** (decoherence, gate & read-out) dominates NISQ performance; modelling with quantum channels allows simulation and mitigation.  
- **Error-mitigation** gives partial relief without full error-correction; ZNE, read-out mitigation, PEC, DD commonly used.  
- **Transpilation** maps & optimises circuits for specific hardware, reducing depth and error accumulation; Qiskit provides multi-level optimisation and noise-adaptive strategies.  
- **AI/DS** pipelines must co-design ansätze and mitigation to keep advantage under realistic noise.

---

## 12) Looking ahead  

- **Next Session:** Phase Estimation & Shor-style factoring — introduces precision-sensitive circuits, thus extending today’s noise toolkit.  
- **Homework 2**: build depolarising + read-out noise model, transpile 4-qubit Grover at levels 0–3, evaluate fidelity, and answer mini-exercise questions.

---

## Appendix — solutions to mini-exercises (sketch)

1. Pure dephasing Kraus: $K_0=\sqrt{1-p}I,\, K_1=\sqrt{p}Z$ with $p=1-e^{-t/T_2}$.  
2. Level-0 depth 35, CX 8; level-3 depth 22, CX 4 (numbers may vary).  
3. Build calibration matrix $A$, invert, apply to raw counts → error rate drops from ~5 % to <1 %.  
4. Success ≈ $(1-p)^{\text{gate count}}$ with ~200 gates → $\approx e^{-1}\approx0.37$.  
5. Effective rotation angle $(2\theta+\delta)$ per iteration ⇒ optimal $k$ decreases by $\delta/(4\theta^2)$ to first order.

